**part1**

In [2]:
%pip install cryptography

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 33.6 MB/s eta 0:00:00


In [3]:
from cryptography.hazmat.primitives.asymmetric import rsa
from sympy.ntheory.modular import crt
from random import randint
from math import log2


class FastRSA:
    def __init__(self):
        private_key = rsa.generate_private_key(
            public_exponent=2**16 + 1,
            key_size=4096,
        )
        private_numbers = private_key.private_numbers()
        public_numbers = private_key.public_key().public_numbers()
        p, q, d = private_numbers.p, private_numbers.q, private_numbers.d
        self.n, self.e = public_numbers.n, public_numbers.e
        self.__dp = d % (p - 1)
        self.__dq = d % (q - 1)
        self.__p = p
        self.__q = q

    def sign(self, m, inject_fault=False):
        m1 = pow(m, self.__dp, self.__p)
        m2 = pow(m, self.__dq, self.__q)
        if inject_fault:
            m2 ^= 1 << randint(0, int(log2(m2)))
        return crt([self.__p, self.__q], [m1, m2])[0]

**part2**

In [4]:
import os , math

In [5]:
def gcdExtended(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = gcdExtended(b % a, a)
        return (g, x - (b // a) * y, y)

In [6]:
def modInverse(a, m):
    g, x, y = gcdExtended(a, m)
    if g != 1:
        raise Exception("Inverse doesn't exist")
    else:
        return x % m

In [7]:
file = open("/content/sig1.txt",'r')

In [8]:
lines = file.readlines()

In [9]:
type(lines) , len(lines)

(list, 4)

In [10]:
lines[0]

'n=9701083685038056759383292130228643028079965784372036392485831776797214688013815733244083542927604401298890273428917432955478266217127720225840691075164231468209198655568491220577265445405010008272208386892953903299007526396924657650051798911855713378661778219657197451530899894392998474793414765002196401252627029413168715017073708318151083147253165799005474831446240111083608368652670975477071316216639850351724516852835070903523529089314221296578302477284622030792205317068888280652253840840887338175767629465282596245282928214367245412855193148655894746373624781452933752792768231046543940286516812404561901710090898895656583928447662513223290667808041159715991758570215363241921115578286902390603375082652514633417283916542505180687826802173635384436095558271637873709536068540530556479401449736699751991259294947327959476045254919550644154105802445541974231550937833355631894225090415775225712887051989800301212881726866471502593391379474328888884294201237348266795445642971960309550816037486

In [34]:
n = int(lines[0][2:len(lines[0])-1])
n

9701083685038056759383292130228643028079965784372036392485831776797214688013815733244083542927604401298890273428917432955478266217127720225840691075164231468209198655568491220577265445405010008272208386892953903299007526396924657650051798911855713378661778219657197451530899894392998474793414765002196401252627029413168715017073708318151083147253165799005474831446240111083608368652670975477071316216639850351724516852835070903523529089314221296578302477284622030792205317068888280652253840840887338175767629465282596245282928214367245412855193148655894746373624781452933752792768231046543940286516812404561901710090898895656583928447662513223290667808041159715991758570215363241921115578286902390603375082652514633417283916542505180687826802173635384436095558271637873709536068540530556479401449736699751991259294947327959476045254919550644154105802445541974231550937833355631894225090415775225712887051989800301212881726866471502593391379474328888884294201237348266795445642971960309550816037486979

In [18]:
lines[1], len(lines[1])

('e=65537\n', 8)

In [54]:
e = int(lines[1][2:len(lines[1])])
e

65537

In [13]:
lines[2]

'm=32182110544320125172886333925353399125487506562333020210884780807854499260343379279075188696785651525982546809357207590252249760987119653409235590450967339449658034838156478667128606165728335268981909308841799044101228980630658166850370245232624955185155355066892278059868749790525947423836740428528253204901\n'

In [55]:
m = int(lines[2][2:len(lines[2])])
m

32182110544320125172886333925353399125487506562333020210884780807854499260343379279075188696785651525982546809357207590252249760987119653409235590450967339449658034838156478667128606165728335268981909308841799044101228980630658166850370245232624955185155355066892278059868749790525947423836740428528253204901

In [14]:
lines[3]

'sig=13971907037488901701926991371208449774539288378353211918429819378808376573200674223848262547259628316429776348955446803426630556565039636559461117070874448594442741672678197045667247152283659657003334646318236653978836941750585713902888360494171706959331905860588582085192338594348518666192283512631833670327029788290686659513321446774782338133955638239335635140651486395260496670233703823042579493322767601302090073404903233769814251881500404669977271216471518018214591163424108262789504139431416084690333130085791165095122984156867164053445505872861413853669158079867980033287903434283786225862452438814005708504170134405898021741822323660864068559638557923782471779652423769452684826191092216692064532846264884227993742615274318666953270655374893600819413872922540105760853131299340165909072391010319615909323180455785771329842109459196317128806735362233384603234049655931851307319704996492897999727155261564559623759175038393079802627180068638562531703171101523727242179000458849308492690852

In [65]:
sign = int(lines[3][4:len(lines[3])])
sign

1397190703748890170192699137120844977453928837835321191842981937880837657320067422384826254725962831642977634895544680342663055656503963655946111707087444859444274167267819704566724715228365965700333464631823665397883694175058571390288836049417170695933190586058858208519233859434851866619228351263183367032702978829068665951332144677478233813395563823933563514065148639526049667023370382304257949332276760130209007340490323376981425188150040466997727121647151801821459116342410826278950413943141608469033313008579116509512298415686716405344550587286141385366915807986798003328790343428378622586245243881400570850417013440589802174182232366086406855963855792378247177965242376945268482619109221669206453284626488422799374261527431866695327065537489360081941387292254010576085313129934016590907239101031961590932318045578577132984210945919631712880673536223338460323404965593185130731970499649289799972715526156455962375917503839307980262718006863856253170317110152372724217900045884930849269085265741

In [73]:
def get_data(file_path):
  file = open(file_path,'r')
  lines = file.readlines()
  n = int(lines[0][2:len(lines[0])])
  e = int(lines[1][2:len(lines[1])])
  m = int(lines[2][2:len(lines[2])])
  sign = int(lines[3][4:len(lines[3])])
  return n , e , m , sign

In [74]:
file1 = "/content/sig1.txt"
file2 = "/content/sig2.txt"
file3 = "/content/sig3.txt"

In [77]:
#checking for sig1.txt
n1, e1, m1, sign1 = get_data(file1)
mm1 = pow(sign1, e1, n1)
if (m1 == mm1 ):
  print("correct signiture")


correct signiture


In [81]:
#checking for sig2.txt
n2, e2, m2, sign2 = get_data(file2)
mm2 = pow(sign2, e2, n2)
if (m2 == mm2 ):
  print("correct signiture")
else:
  print("Incorrect signiture")


correct signiture


In [82]:
#checking for sig3.txt
n3, e3, m3, sign3 = get_data(file3)
mm3 = pow(sign3, e3, n3)
if (m3 == mm3 ):
  print("correct signiture")
else:
  print("Incorrect signiture")

Incorrect signiture


paper's approach

In [95]:
n3, e3, m3, sign3 = get_data(file3)


In [102]:
q = math.gcd(n3, (pow(sign3, e3, n3) - m3))

In [108]:
q

31341229038749314244797258919637636916441955906836402672971618438862383882051262592032888744453189634620931974910432931959705675289497166109180515325497674836933008331605908916257639917312169415006076453976468416277666276876749704285008291437207547081351658150492520813841414148957843427724035033708078507670808416897314599702277061732558753670772498366238932710726306979137055202882348778279150697691262849111945632593615519075513477823782408815406831006117927964077578537687042065535210335664166638806623619081487668217300247032276633706168397935896909478271849375578808644196740939370935488580037218350664830977701

In [103]:
p = n3 // q

In [110]:
p

30953105486207770110128938520936870655523023196433068034361051671457763190570433443983536656599259247450895709471949832215474337214219697438157281692653635924390335831720768295296630438389448147052719540138246132904917568013366050232950641858739777063386433253172579014592368010200148490857617749225138818318842629508080340102113234869562868928160941570045016777455228225644617466929471220796423517282307877405375087266985112876867349485987537356060416253221135843057506100224740988203274140265042431656267383286508144590889120933361011809369717673787018046408502901215802100543929292065468690675654619841973561378829

In [104]:
fi3 = (p-1)*(q-1)

In [111]:
fi3

9701083685038056759383292130228643028079965784372036392485831776797214688013815733244083542927604401298890273428917432955478266217127720225840691075164231468209198655568491220577265445405010008272208386892953903299007526396924657650051798911855713378661778219657197451530899894392998474793414765002196401252627029413168715017073708318151083147253165799005474831446240111083608368652670975477071316216639850351724516852835070903523529089314221296578302477284622030792205317068888280652253840840887338175767629465282596245282928214367245412855193148655894746373624781452933752792768231046543940286516812404561901710090275952311334357604113251248884922732321509924959063863142036540817914107560685430243210828641990144596565639698681353046075002048598215800622180301456360601922835098897289707285907033142735815638706987386812330553429081101742996560622856209014758309490452441595243226806077953634132967866173272471880708466969961038539441981430425922863077975248013867432606148090144957502999310788861

In [112]:
d3 = modInverse(e3, fi3)

In [113]:
d3

5137043322177712159202248654766846630857181936628883698747704441490616575870599221912853430485978655456866930879917460263468235512751581590820076339663083279258007387320886206553754673197361297085291054804661065658921787632617808552228475966813260861697641810503736520712396813021874957188010833645669223630486113626723943512100431412379498444272302148232082618223451131651518147393415834306670780749534909541270543858595729139812328204152779893441161621247318659026392622847531911618716408937579599066967359094300429072070719452391792190788815829698554576470547544372531744768912655297606861829245761290384305612813722578833522247681357802025440626798640244143549130297488155333819749014889085969317490709022256526512950149688008875537650287182729640983639656151208348540963578883258793383915194740042807936675857718392235456605065883860336740354911814728743277421495592742010182354137023807969833079280828802781087753584047568974494908136221699211545237927476190140765997280365600967471567024453616